# Building a Machine Learning workflow ([video #10](https://www.youtube.com/watch?v=irHhDMbw3xo&list=PL5-da3qGB5ICeMbQuqbbCOQWcS6OYBr5A&index=10))

Created by [Data School](https://www.dataschool.io). Watch all 10 videos on [YouTube](https://www.youtube.com/playlist?list=PL5-da3qGB5ICeMbQuqbbCOQWcS6OYBr5A). Download the notebooks from [GitHub](https://github.com/justmarkham/scikit-learn-videos).

**Note:** This notebook uses Python 3.9.1 and scikit-learn 0.23.2. The original notebook (shown in the video) used Python 3.7 and scikit-learn 0.20.2.

## Agenda

- Why should you use a Pipeline?
- How do you encode categorical features with OneHotEncoder?
- How do you apply OneHotEncoder to selected columns with ColumnTransformer?
- How do you build and cross-validate a Pipeline?
- How do you make predictions on new data using a Pipeline?
- Why should you use scikit-learn (rather than pandas) for preprocessing?

## Step 1: Load the dataset

In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('http://bit.ly/kaggletrain')

In [24]:
df.shape

(891, 12)

## Step 2: Select features

In [25]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [26]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [27]:
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]

In [28]:
df.shape

(889, 4)

In [29]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [30]:
df.head()

Survived  Pclass     Sex Embarked
0         0       3    male        S
1         1       1  female        C
2         1       3  female        S
3         1       1  female        S
4         0       3    male        S

## Step 3: Cross-validate a model with one feature

In [31]:
X = df.loc[:, ['Pclass']]
y = df.Survived

In [32]:
X.shape

(889, 1)

In [33]:
y.shape

(889,)

In [34]:
from sklearn.linear_model import LogisticRegression

In [35]:
logreg = LogisticRegression()

In [36]:
from sklearn.model_selection import cross_val_score

In [37]:
cross_val_score(logreg, X, y, cv=5, scoring='accuracy').mean()

np.float64(0.6783406335301212)

In [38]:
y.value_counts(normalize=True)

Survived
0    0.617548
1    0.382452
Name: proportion, dtype: float64

## Step 4: Encode categorical features

In [39]:
df.head()

Survived  Pclass     Sex Embarked
0         0       3    male        S
1         1       1  female        C
2         1       3  female        S
3         1       1  female        S
4         0       3    male        S

In [40]:
# dummy encoding of categorical features
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)

In [41]:
ohe.fit_transform(df[['Sex']])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], shape=(889, 2))

In [42]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [43]:
ohe.fit_transform(df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]], shape=(889, 3))

In [44]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

## Step 5: Cross-validate a Pipeline with all features

In [45]:
X = df.drop('Survived', axis='columns')

In [46]:
X.head()

Pclass     Sex Embarked
0       3    male        S
1       1  female        C
2       3  female        S
3       1  female        S
4       3    male        S

In [47]:
# use when different features need different preprocessing
from sklearn.compose import make_column_transformer

In [48]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),
    remainder='passthrough')

In [49]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]], shape=(889, 6))

In [50]:
# chain sequential steps together
from sklearn.pipeline import make_pipeline

In [51]:
pipe = make_pipeline(column_trans, logreg)

In [52]:
# cross-validate the entire process
# thus, preprocessing occurs within each fold of cross-validation
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

np.float64(0.7727924839713071)

## Step 6: Make predictions on "new" data

In [53]:
# added empty cell so that the cell numbering matches the video

In [54]:
X_new = X.sample(5, random_state=99)
X_new

Pclass     Sex Embarked
599       1    male        C
512       1    male        S
273       1    male        C
215       1  female        C
790       3    male        Q

In [55]:
pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [56]:
pipe.predict(X_new)

array([1, 0, 1, 1, 0])

## Recap

In [57]:
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [58]:
df = pd.read_csv('http://bit.ly/kaggletrain')
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]
X = df.drop('Survived', axis='columns')
y = df.Survived

In [59]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),
    remainder='passthrough')
logreg = LogisticRegression(solver='lbfgs')

In [60]:
pipe = make_pipeline(column_trans, logreg)

In [61]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

np.float64(0.7727924839713071)

In [62]:
X_new = X.sample(5, random_state=99)

In [63]:
pipe.fit(X, y)
pipe.predict(X_new)

array([1, 0, 1, 1, 0])